Name: Smita Gupta<br>
USC ID: 4123927261

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras
import sys
import copy
import string
import os
from sklearn.preprocessing import MinMaxScaler
import operator
from sklearn.cluster import KMeans
from skimage.color import rgb2gray
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

1. Generative Models for Text<br><br>
(a) In this problem, we are trying to build a generative model to mimic the writ- ing style of prominent British Mathematician, Philosopher, prolific writer, and political activist, Bertrand Russell.<br>
<br>(b) Download the following books from Project Gutenberg http://www.gutenberg. org/ebooks/author/355 in text format:<br>
i. The Problems of Philosophy<br>
ii. The Analysis of Mind<br>
iii. Mysticism and Logic and Other Essays<br>
iv. Our Knowledge of the External World as a Field for Scientific Method in Philosophy<br>
Project Gutenberg adds a standard header and footer to each book and this is not part of the original text. Open the file in a text editor and delete the header and footer.<br>
The header is obvious and ends with the text:<br>
*** START OF THIS PROJECT GUTENBERG EBOOK AN INQUIRY INTO MEANING AND TRUTH ***<br>
The footer is all of the text after the line of text that says:<br>
THE END<br>
To have a better model, it is strongly recommended that you download the fol- lowing books from The Library of Congress https://archive.org and convert them to text files:<br>
i. The History of Western Philosophy<br>
https://archive.org/details/westernphilosophy4<br>
ii. The Analysis of Matter<br>
https://archive.org/details/in.ernet.dli.2015.221533<br>
iii. An Inquiry into Meaning and Truth<br>
https://archive.org/details/BertrandRussell-AnInquaryIntoMeaningAndTruth<br>
Try to only use the text of the books and throw away unwanted text before and after the text, although in a large corpus, these are considered as noise and should not make big problems.<br>
<br>(c) LSTM: Train an LSTM to mimic Russell’s style and thoughts:<br>
i. Concatenate your text files to create a corpus of Russell’s writings.<br>

In [4]:
#Setting Path
books_path = "books/"

In [5]:
dataFiles = os.listdir(books_path)
resultFile = open("CORPUS.txt", 'w')
for fileItem in dataFiles:
    with open(books_path + fileItem, 'r', 
              encoding='ascii', errors='ignore') as fileDataStream:
        for line in fileDataStream:
            resultFile.write(line)

resultFile.close()
resultFile = resultFile.name

In [6]:
contents = open(resultFile, 'r').read()
print("The length of corpus of Russell's writing is: ", len(contents))

The length of corpus of Russell's writing is:  5095252


1) c) ii. Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0, 1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers as its input.

In [35]:
short_text = copy.copy(contents)
character_set = set(contents)
short_text = contents.lower()
character_set = set(contents.lower())
character_set = character_set.difference(set(string.punctuation))
short_text = short_text.translate(str.maketrans('', '', string.punctuation))

trimmedContent = short_text
CharacterSet = character_set

In [36]:
dict_char2Ascii = dict()
for index, char in enumerate(sorted(CharacterSet)):
    dict_char2Ascii[char] = index

print("Original Dictionary:\n", sorted(dict_char2Ascii.items(), key=operator.itemgetter(1)))
dict_scaled_char = dict()
scaledValues = MinMaxScaler().fit_transform(np.array(list(dict_char2Ascii.values())).reshape(-1, 1))
for index in range(len(scaledValues)):
    dict_scaled_char[list(dict_char2Ascii.keys())[index]] = scaledValues[index][0]

print("\nScaled Dictionary:\n", sorted(dict_scaled_char.items(), key=operator.itemgetter(1)))

ascii2CharDict = {v:k for k, v in dict_char2Ascii.items()}
print("\nReversed Dictionary:\n", sorted(ascii2CharDict.items(), key=operator.itemgetter(1)))

Original Dictionary:
 [('\n', 0), (' ', 1), ('0', 2), ('1', 3), ('2', 4), ('3', 5), ('4', 6), ('5', 7), ('6', 8), ('7', 9), ('8', 10), ('9', 11), ('a', 12), ('b', 13), ('c', 14), ('d', 15), ('e', 16), ('f', 17), ('g', 18), ('h', 19), ('i', 20), ('j', 21), ('k', 22), ('l', 23), ('m', 24), ('n', 25), ('o', 26), ('p', 27), ('q', 28), ('r', 29), ('s', 30), ('t', 31), ('u', 32), ('v', 33), ('w', 34), ('x', 35), ('y', 36), ('z', 37)]

Scaled Dictionary:
 [('\n', 0.0), (' ', 0.02702702702702703), ('0', 0.05405405405405406), ('1', 0.08108108108108109), ('2', 0.10810810810810811), ('3', 0.13513513513513514), ('4', 0.16216216216216217), ('5', 0.1891891891891892), ('6', 0.21621621621621623), ('7', 0.24324324324324326), ('8', 0.2702702702702703), ('9', 0.2972972972972973), ('a', 0.32432432432432434), ('b', 0.35135135135135137), ('c', 0.3783783783783784), ('d', 0.40540540540540543), ('e', 0.43243243243243246), ('f', 0.4594594594594595), ('g', 0.4864864864864865), ('h', 0.5135135135135136), ('i', 0.

1.) c) iii) Choose a window size, e.g., W = 100.<br> <br>
1.) c) iv) Inputs to the network will be the first W −1 = 99 characters of each sequence, and the output of the network will be the Wth character of the sequence. Basically, we are training the network to predict each character using the 99 characters that precede it. Slide the window in strides of S = 1 on the text. For example, if W = 5 and S = 1 and we want to train the network with the sequence ABRACADABRA, The first input to the network will be ABRA and the corresponding output will be C. The second input will be BRAC and the second output will be A, etc.

In [37]:
win_size = 99
X_data = []
Y_char_data = []
for i in range(0, len(trimmedContent) - win_size, 1):
    tempBlock = trimmedContent[i:i + win_size]
    theLastChar = trimmedContent[i + win_size]
    X_data.append([dict_scaled_char[char] for char in tempBlock])
    Y_char_data.append(dict_char2Ascii[theLastChar])

1.) c) v) Note that the output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements. This means that the network reads integers, but outputs a vector of N = 256 (or less) elements.

In [38]:
numBlocks = len(X_data)
X_data = np.reshape(X_data, (numBlocks, win_size, 1))
Y_data = keras.utils.to_categorical(Y_char_data)
print(X_data.shape)
print(Y_data.shape)

(4942256, 99, 1)
(4942256, 38)


1.) c) vi) Use a single hidden layer for the LSTM with N = 256 (or less) memory units.<br>
1.) c) vii) Use a Softmax output layer to yield a probability prediction for each of the
characters between 0 and 1. This is actually a character classification problem
with N classes. Choose log loss (cross entropy) as the objective function for
the network (research what it means).<br>
1.) c) viii) We do not use a test dataset. We are using the whole training dataset to learn the probability of each character in a sequence. We are not seeking for a very accurate model. Instead we are interested in a generalization of the dataset that can mimic the gist of the text.

In [34]:
mem_unit = 256

model = keras.models.Sequential([
    keras.layers.LSTM(units=mem_unit,
                      input_shape=(X_data.shape[1], X_data.shape[2])),
    keras.layers.Dense(Y_data.shape[1], activation="softmax"),
])
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_3 (Dense)              (None, 38)                9766      
Total params: 305,702
Trainable params: 305,702
Non-trainable params: 0
_________________________________________________________________


1.) c) ix) Choose a reasonable number of epochs4 for training, considering your compu- tational power (e.g., 30, although the network will need more epochs to yield a better model).

In [15]:
epochs = 30
output_dir = "./text_generation"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix = os.path.join(output_dir, 'ck_{epoch:02d}.hdf5')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    monitor='loss',
    save_weights_only= True,
    mode='min'
)
earlyStoppingCk = keras.callbacks.EarlyStopping(patience = 5, min_delta = 1e-4)
history = model.fit(X_data, Y_data, epochs=epochs, batch_size=128, 
                    callbacks=[checkpoint_callback, earlyStoppingCk])

Epoch 1/30
4942256/4942256 [==============================] - 444s 90us/sample - loss: 2.6819
Epoch 2/30
4942256/4942256 [==============================] - 439s 89us/sample - loss: 2.5241
Epoch 3/30
4942256/4942256 [==============================] - 439s 89us/sample - loss: 2.4181
Epoch 4/30
4942256/4942256 [==============================] - 440s 89us/sample - loss: 2.3259
Epoch 5/30
4942256/4942256 [==============================] - 439s 89us/sample - loss: 2.2497
Epoch 6/30
4942256/4942256 [==============================] - 440s 89us/sample - loss: 2.1865
Epoch 7/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.1323
Epoch 8/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.0826
Epoch 9/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.0390
Epoch 10/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.0018
Epoch 11/30
4942256/4942256 [==============================

1.) c) x) Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss.

In [17]:
min_loss_point = output_dir +'/ck_30.hdf5'
model.load_weights(min_loss_point)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))

1.) c) xi) Use the network with the best weights to generate 1000 characters, using the following text as initialization of the network:<br>
There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.

In [19]:
init_text = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'
init_text = init_text.translate(str.maketrans('', '', string.punctuation))

In [26]:
genText = copy.copy(init_textinitText.lower())
encodeList = [dict_scaled_char[char] for char in genText][-99:]
for _ in range(1000):
    data = np.reshape(encodeList, (1, len(encodeList), 1))
    pred = model.predict(data)
    charIndex = np.argmax(pred)
    char = ascii2CharDict[np.argmax(pred)]
    genText += char
    encodeList.append(dict_scaled_char[char])
    encodeList = encodeList[1:len(encodeList)]
    
print(genText)

there are those who take mental phenomena naively just as they would physical phenomena this school of psychologists tends not to emphasize the object much exposure to content there  to be any way to control of the semse and the semse of the 
perception of the semse in the sense of the soace and the 
content of the soace and the soace and the soace and the soace of the content of the soace 
of the soace and the soace and the soace and the soace and the 



170  





















































































































































































































































































































































































































































































































































